# Module 3: The Container (File Formats)
**Goal**: Shatter the illusion that "data is just text." We will prove that how you package bytes (Text vs. Binary, Row vs. Column) dramatically impacts speed and storage, dictated by the physics of CPU parsing and I/O alignment.

---

### 1. Setup and Initialization
First, let's load our physics lab tools. We will use pandas for handling general data, duckdb for high-performance file introspection, and fastavro/pyarrow to manipulate binary formats.

In [ ]:
import pandas as pd
import duckdb
import time
import os
import json
import fastavro
import pyarrow as pa
import pyarrow.csv as pa_csv
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns

# Configure Visualization
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# Define Data Paths (From our pre-loaded Universe)
DATA_DIR = "../data/"
CSV_FILE = f"{DATA_DIR}users.csv"
PARQUET_FILE = f"{DATA_DIR}users.parquet"

# Create a JSON version of users for the experiment
print("Generating lab data...")
df = pd.read_csv(CSV_FILE)
JSON_FILE = f"{DATA_DIR}users.json"
df.to_json(JSON_FILE, orient="records", lines=True)

print("Setup Complete. Lab is ready.")

---

### 2. Experiment 3.1: The Cost of Text (Schema-on-Read)
**The Physics**: Humans love text (CSV/JSON) because we can read it. Computers hate text. When a database reads a CSV, it must perform Parsing:

1. Read a byte.
2. Is it a comma? If no, keep buffering.
3. Is it a newline? If yes, end the row.
4. Convert the string "12345" into the integer 12345 (ASCII-to-Integer conversion).

Binary formats (like Parquet) map directly to memory. The computer doesn't "read" the number; it just copies the bytes.

**Hypothesis**: Will reading a binary format (Parquet) be significantly faster than reading text formats (CSV/JSON), even for the exact same data?

**The Experiment**: We will time how long it takes to load the users dataset into memory using three different formats.

In [ ]:
# 1. Define the experiment function
def benchmark_read(file_path, reader_func, format_name):
    start_time = time.time()
    # Force a materialization to memory (list or df) to trigger the read
    _ = reader_func(file_path)
    end_time = time.time()
    return end_time - start_time

# 2. Run the experiment
results = {}

# Measure CSV (Text)
results['CSV'] = benchmark_read(CSV_FILE, pd.read_csv, 'CSV')

# Measure JSON (Text + Structure overhead)
results['JSON'] = benchmark_read(JSON_FILE, lambda f: pd.read_json(f, lines=True), 'JSON')

# Measure Parquet (Binary + Columnar)
results['Parquet'] = benchmark_read(PARQUET_FILE, pd.read_parquet, 'Parquet')

# 3. Visualize
plt.bar(results.keys(), results.values(), color=['salmon', 'orange', 'skyblue'])
plt.title("Read Time: Text vs. Binary (Lower is Better)")
plt.ylabel("Time (Seconds)")
plt.show()

print(f"CSV Time: {results['CSV']:.4f}s")
print(f"Parquet Time: {results['Parquet']:.4f}s")
print(f"Speedup Factor: {results['CSV'] / results['Parquet']:.1f}x faster")

**The Conclusion**: You likely observed that Parquet was the fastest and that JSON was the slowest.
- **CSV/JSON (Schema-on-Read)**: The CPU spent most of its time decoding text (finding commas, converting "1" "0" "0" to integer 100). This is CPU-bound.
- **Parquet (Schema-on-Write)**: The schema is stored in the file header. The engine knows exactly how many bytes to grab for an integer. It simply `memcpy` (memory copy) the data from disk to RAM.

---

### 3. Experiment 3.2: The Power of Metadata (Data Skipping)
**The Physics**: In a CSV, if you want to find orders from "2023-01-01", you must scan the file from top to bottom because the file has no "Brain"—it's just text.

Columnar formats like Parquet divide rows into Row Groups. Each group has a Footer containing statistics (Min/Max values) for the columns in that chunk. If you ask for WHERE order_date = '2025-01-01', the database looks at the footer first. If the footer says "This chunk contains dates from 2020 to 2022", the database skips the entire chunk without reading the data.

**Hypothesis**: If we filter for a value that exists at the very end of the file, Parquet will be instant because it skips the early data, while CSV will be slow because it must scan everything.

**The Experiment**: We will use the orders_sorted.csv data. Since it is sorted by date, Parquet's Min/Max statistics will be perfectly efficient (chunks will not overlap).

In [ ]:
# 1. Prepare Data: Convert sorted CSV to Parquet with small Row Groups
# We force small row groups to create many "chunks" for the demo
print("Preparing partitioned Parquet file...")
table = pa.csv.read_csv(f"{DATA_DIR}orders_sorted.csv")
pq.write_table(table, f"{DATA_DIR}orders_sorted_chunked.parquet", row_group_size=10000)

# 2. Define the Query
# We pick a date that we know is at the END of the file
target_date = "2023-12-31" 
query_csv = f"SELECT * FROM '{DATA_DIR}orders_sorted.csv' WHERE order_date = '{target_date}'"
query_parquet = f"SELECT * FROM '{DATA_DIR}orders_sorted_chunked.parquet' WHERE order_date = '{target_date}'"

# 3. Run the Experiment
times = {}

# CSV Scan
start = time.time()
duckdb.sql(query_csv).fetchall()
times['CSV (Scan)'] = time.time() - start

# Parquet Scan (Smart)
start = time.time()
duckdb.sql(query_parquet).fetchall()
times['Parquet (Pruning)'] = time.time() - start

# 4. Visualize
plt.bar(times.keys(), times.values(), color=['red', 'green'])
plt.title("Query Time: Full Scan vs. Partition Pruning")
plt.ylabel("Time (Seconds)")
plt.show()

# Bonus: Show the Metadata that made this possible
print("\n--- The Cheat Sheet (Parquet Metadata) ---")
print(duckdb.sql(f"SELECT row_group_id, stats_min, stats_max FROM parquet_metadata('{DATA_DIR}orders_sorted_chunked.parquet') WHERE path_in_schema = 'order_date' LIMIT 5").df())

**The Conclusion**: The Parquet query was likely near-instant, while the CSV query took measurable time.
- **The Physics**: The CSV engine had to parse 500,000 dates to check if they matched. The Parquet engine read the Metadata Footer first. It saw that the first 49 chunks had `MAX(order_date) < 2023-12-31`, so it physically did not read those bytes from the disk. It only jumped to the last chunk.

----

### 4. Experiment 3.3: Serialization (Row-Based vs. Column-Based Writing)
**The Physics**: Not all binary is created equal.
- **Avro (Row-Based Binary)**: Stores data physically as [Row1_Bytes][Row2_Bytes]. This is perfect for Streaming (Kafka) because you can write one row at a time instantly.
- **Parquet (Column-Based Binary)**: Stores data physically as [Col1_All_Rows][Col2_All_Rows]. To write a Parquet file, you must buffer many rows in memory, pivot them, and then write the block. It is terrible for writing one row at a time.

**Hypothesis**: Simulating a streaming producer (writing 1 row at a time) will be much faster with Avro/Row-based formats than Parquet.

**The Experiment**: We will simulate a "Log Producer" writing 1,000 events, one by one, to disk.

In [ ]:
# 1. Setup Dummy Data (1000 records)
records = [{"id": i, "status": "active", "ts": time.time()} for i in range(1000)]
schema = {
    "type": "record", "name": "User", "fields": [
        {"name": "id", "type": "int"},
        {"name": "status", "type": "string"},
        {"name": "ts", "type": "double"}
    ]
}

# 2. Define Writers

def write_avro_stream():
    # Avro NATIVELY supports streaming.
    # We pass the list (iterator), and it writes Row 1, then Row 2, etc. efficiently.
    with open(f"{DATA_DIR}stream.avro", "wb") as out:
        fastavro.writer(out, schema, records)

def write_parquet_stream():
    # Parquet DOES NOT support streaming rows naturally.
    # To simulate a live stream (writing data as soon as it arrives),
    # we are forced to convert EVERY row into a mini-table and write it.
    # This simulates the "Overhead" of using a Columnar format for real-time data.
    for record in records:
        table = pa.Table.from_pylist([record])
        pq.write_table(table, f"{DATA_DIR}stream_temp.parquet")

# 3. Run Benchmark
stream_times = {}

print("Streaming Avro (Row-by-Row)...")
start = time.time()
write_avro_stream()
stream_times['Avro (Row)'] = time.time() - start

print("Streaming Parquet (Forced Row-by-Row)...")
start = time.time()
write_parquet_stream()
stream_times['Parquet (Col)'] = time.time() - start

# 4. Visualize
# We use Log Scale because the difference is usually massive
plt.bar(stream_times.keys(), stream_times.values(), color=['gold', 'purple'])
plt.yscale('log') 
plt.title("Streaming Write Speed: 1000 Rows (Log Scale)")
plt.ylabel("Time (Seconds)")
plt.show()

print(f"Avro Time: {stream_times['Avro (Row)']:.4f}s")
print(f"Parquet Time: {stream_times['Parquet (Col)']:.4f}s")

**The Conclusion**: Parquet is likely orders of magnitude slower here.
- **Avro**: Is a "Streaming" format. It dumps the bytes for the row and moves on.
- **Parquet**: Is a "Storage" format. For every single write, it has to calculate metadata, encode columns, and organize headers. It is too heavy for real-time row processing.
- **Lesson**: Use Avro/Protobuf for Moving data (Kafka/APIs). Use Parquet/ORC for Analyzing data (Data Lakes).

---